# Adding Moderation

This moderation is a copied from this [github repo from Pere Martra](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/LangChain_OpenAI_Moderation_System.ipynb). The Langchain cookbook moderation fails due to the update OpenAI policy

In [1]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
#PrompTemplate is a custom class that provides funcrionality to create prompts
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [4]:

assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
User:" {customer_request}"
Comment:
"""

In [6]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

In [7]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False,
)
#the output of the formatted prompt will pass directly to the LLM.

In [8]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #calling the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

In [9]:
# This is the customer request, or customer comment in the forum moderated by the agent.
# feel free to modify it.
customer_request = """This product is a piece of shit. I feel like an Idiot!"""

In [10]:
# Our assistant working in 'nice' mode.
response_data=create_dialog(customer_request, "nice")
print(f"assistant response: {response_data['assistant_response']}")

assistant response: I totally understand your frustration with this product. It can be really disappointing when something doesn't live up to our expectations.


In [11]:
#Our assistant running in rude mode.
response_data = create_dialog(customer_request, "very rude")
print(f"assistant response: {response_data['assistant_response']}")

assistant response: Well, it seems like you have quite the negative opinion about this product. It appears to be causing you quite a bit of frustration and disappointment.


## Moderator

In [12]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [13]:
#I'm going to use a more advanced LLM
moderator_llm = ChatOpenAI(model="gpt-3.5-turbo")

In [14]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [15]:
# To run our chain we use the .run() command
moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})

In [16]:
print(moderator_data['text'])


Comment: {'customer_request': 'This product is a piece of shit. I feel like an Idiot!', 'sentiment': 'very rude', 'assistant_response': 'Well, it seems like you have quite a negative opinion about this product. It appears to be causing you quite a bit of frustration and disappointment.'}


### LangChain System

In [17]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

In [18]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True
)

In [21]:
# We can now run the chain.
assistant_moderated_chain.invoke({"sentiment": "polite", "customer_request": customer_request})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: I understand that you're feeling frustrated with this product and it's making you question your decision. I'm here to help, so please let me know how I can assist you in resolving any issues you may be facing.


> Finished chain.

> Finished chain.


{'sentiment': 'polite',
 'customer_request': 'This product is a piece of shit. I feel like an Idiot!',
 'text': "I understand that you're feeling frustrated with this product and it's making you question your decision. I'm here to help, so please let me know how I can assist you in resolving any issues you may be facing."}

In [22]:
# We can now run the chain.
assistant_moderated_chain.invoke({"sentiment": "polite", "customer_request": customer_request})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: I understand that you're disappointed with this product and feel frustrated. I'm sorry to hear that you're not satisfied with your purchase. Is there anything specific that I can assist you with or any feedback you'd like to share?


> Finished chain.

> Finished chain.


{'sentiment': 'polite',
 'customer_request': 'This product is a piece of shit. I feel like an Idiot!',
 'text': 'I appreciate your understanding and recognition of the disappointment and frustration that the product has caused you. I apologize for any inconvenience this may have caused and I would like to offer my assistance in any way possible. Please let me know if there is anything specific I can do to help or if you have any feedback you would like to share.'}